# Monitoring coastal erosion along Africa's coastline

* **Products used:**  
[ls8_sr](https://explorer.digitalearth.africa/ls8_sr), [ls9_sr](https://explorer.digitalearth.africa/ls9_sr)  
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)  
[s1_rtc](https://explorer.digitalearth.africa/s1_rtc)

## Background
Over 40% of the world’s population lives within 100 km of the coastline. 
However, coastal environments are constantly changing, with erosion and coastal change presenting a major challenge to valuable coastal infrastructure and important ecological habitats. 
Up-to-date data on coastal change and erosion is essential for coastal managers to be able to identify and minimise the impacts of coastal change and erosion.  

Imagery from satellites such as the NASA/USGS Landsat, Copernicus Sentinel-1/2 program is available for free for the entire planet, making satellite imagery a powerful and cost-effective tool for monitoring coastlines and rivers at regional or national scale.

### DE Africa use case
The usefulness of optical imagery such as Landsat and Sentinel-2 in the coastal zone can be affected by the presence of clouds, sun-glint over water poor water quality (e.g. sediment) and the influence of tides. The effect of these factors can be reduced by combining individual noisy images into cleaner "summary" or composite layers, and filtering the data to focus only on images taken at certain tidal conditions (e.g. mid-tide). These clean, tidally-constrained composite images can then be used to identify and extract the precise boundary between water and land. This allows us to extract accurate shorelines that can be compared across time to reveal hotspots of erosion and coastal change. 

Radar observations are largely unaffected by cloud cover, so can take reliable measurements of areas in any weather. Radar data is readily available from the ESA/EC Copernicus program’s Sentinel-1 satellites. The two satellites provide all-weather observations, with a revisit time of 6 days. By developing a process to classify the observed pixels as either water or land, it is possible to identify the shoreline from radar data.

### Description

In this example, we use a simplified version of the [DE Africa Coastlines](https://github.com/digitalearthafrica/deafrica-coastlines.git) method to combine data from Landsat/Sentinel-1/Sentinel-2 satellite with image compositing and tide filtering techniques to accurately map shorelines across time, and identify areas that have changed significantly between 2018 and 2021. 
The worked example demonstrates how to:
1. Query the satellite data and select best available product
2. Process the selected data and generate annual composite images for each year
3. Extract shorelines and calculate rates of coastal change  

## Getting started
**To run this analysis**, run all the cells in the notebook, starting with the "Load packages" cell.

**After finishing the analysis**, return to the "Analysis parameters" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.

### Load packages
First we need to install additional tools from the [DE Africa Coastlines](https://github.com/digitalearthafrica/deafrica-coastlines.git) repository that will allow us to estimate rates of coastal change. 
> **Note:** If you run into any error messages in this analysis, try restarting the notebook by clicking `Kernel`, then `Restart Kernel and Clear All Outputs`.

In [6]:
pip install -q git+https://github.com/digitalearthafrica/deafrica-coastlines.git@S2_test --disable-pip-version-check

In [2]:
# pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@coastlines_s2s1#subdirectory=Tools

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git (to revision coastlines_s2s1) to /tmp/pip-req-build-m8g4d2jh
  Running command git clone --filter=blob:none --quiet https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git /tmp/pip-req-build-m8g4d2jh
  Running command git checkout -b coastlines_s2s1 --track origin/coastlines_s2s1
  Switched to a new branch 'coastlines_s2s1'
  Branch 'coastlines_s2s1' set up to track remote branch 'coastlines_s2s1' from 'origin'.
  Resolved https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git to commit 16958c41755439799b561a68dc7ee43db563dad1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install /home/jovyan/deafrica-sandbox-notebooks/Tools

Defaulting to user installation because normal site-packages is not writeable
Processing /home/jovyan/deafrica-sandbox-notebooks/Tools
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for deafrica-tools: filename=deafrica_tools-1.0.3-py3-none-any.whl size=146344 sha256=f31ab4aee980132ff7886524a77e1d98204e49baecd31f32479c003a4e296846
  Stored in directory: /tmp/pip-ephem-wheel-cache-7vqrh7yd/wheels/f2/1c/be/b1b8ca673f3c7502ee9cf5ec4e0a4cbd63485a4772d1c5b338
Successfully built deafrica-tools
  Attempting uninstall: deafrica-tools
    Found existing installation: deafrica-tools 1.0.3
    Uninstalling deafrica-tools-1.0.3:
      Successfully uninstalled deafrica-tools-1.0.3
Note: you may need to restart the kernel to use updated packages.


Now we can load key Python packages and supporting functions for the analysis.

In [2]:
%matplotlib inline

# Force GeoPandas to use Shapely instead of PyGEOS
# In a future release, GeoPandas will switch to using Shapely by default.
import os
os.environ['USE_PYGEOS'] = '0'

import datacube
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import nearest_points

from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.coastal import pixel_tides
from deafrica_tools.spatial import subpixel_contours
from deafrica_tools.plotting import display_map, rgb, map_shapefile
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_best_available_ds
from deafrica_tools.datahandling import mostcommon_crs
from deafrica_tools.datahandling import load_ard
from coastlines.raster import tide_cutoffs,load_tidal_subset
from coastlines.vector import points_on_line, annual_movements, calculate_regressions

from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_minimum, threshold_otsu
from skimage.morphology import binary_erosion,binary_dilation,disk
from datacube.utils.cog import write_cog

from datetime import datetime
from dateutil import parser

import warnings
warnings.filterwarnings("ignore")

### Set up a Dask cluster

Dask can be used to better manage memory use down and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/08_parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [3]:
client = create_local_dask_cluster(return_client=True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35211,Workers: 1
Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:35499,Total threads: 4
Dashboard: /user/whusggliuqx@gmail.com/proxy/35575/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:32771,


### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [4]:
dc = datacube.Datacube(app="Coastal_erosion_Landsat_Sentinel")

### Analysis parameters

The following cell set important parameters for the analysis:

* `lat`: The central latitude to analyse (e.g. `14.283`).
* `lon`: The central longitude to analyse (e.g. `-16.921`).
* `buffer`: The number of square degrees to load around the central latitude and longitude. For reasonable loading times, set this as `0.1` or lower.
* `time_range`: The date range to analyse (e.g. `('2018', '2021') `)
* `time_step`: This parameter allows us to choose the length of the time periods we want to compare: e.g. shorelines for each year, or shorelines for each six months etc. 
`1Y` will generate one coastline for every year in the dataset; `2Y` will produce a coastline for every two years, etc.
* `filter_size`: An integer number of speckle filtering size in pixels for Sentinel-1, e.g. 2. Set it to None to disable speckle filtering.  

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example explores coastal change in Ponto, Senegal. 

**To run the notebook for a different area**, make sure at least one of Landsat, Sentinel-2 or Sentinel-1 data is available for the new location, which you can check at the [DE Africa Explorer](https://explorer.digitalearth.africa/).

To ensure that the tidal modelling part of this analysis works correctly, please make sure the **centre of the study area is located over water** when setting `lat_range` and `lon_range`.


In [5]:
# Define the area of interest
lat = -12.4
lon = 43.736
# lat = -17.474
# lon = 43.924

buffer =  0.04

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat - buffer, lat + buffer)
lon_range = (lon - buffer, lon + buffer)

# Set the range of dates for the analysis, time step and tide range
time_range = ('2018', '2021')
time_step = '1Y'

# Lee filtering size
filter_size=2

### View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [6]:
display_map(x=lon_range, y=lat_range)

## Query best available product
The first step in this analysis is to select from the available satellite imagery products (Landsat, Sentinel-2, Sentinel-1 or merged Landsat and Sentinel-2) and load the best available one for shoreline mapping at the given location and study period.
Here we define a function `load_best_available_ds` to select and load the data. The function implements a few steps as below:
1. Query all three products using the `lat_range`, `lon_range` and `time_range` specified above and the `load_ard` function. For more information see the [Using load_ard notebook](../Frequently_used_code/Using_load_ard.ipynb). The `load_ard` function will also automatically mask out clouds from the dataset, allowing us to focus on pixels that contain useful data.  
2. Calculate the average number and frequency of obseravations within each timestep. Optionally the calculation can be restricted to a coastal zone instead of the full image extent.   
3. Check if minumum requirement on the number and frequency of observations requirement for shoreline mapping has been met for all products. Select the one with highest number and frequency of observations and return the queried dataset and product name.

This function takes in the `dc`, `lat_range`, `lon_range`, `time_range` and `time_step` as specified above. Other optional parameters include:
* `combine_ls_s2`: A boolean value indicating whether to include merged/stacked Landsat and Sentinel-2 products as an option. By default this parameters is not set, i.e. only Landsat, Sentinel-2 and Sentinel-1 will be queried.
* `coastal_masking`: A boolean value indicating whether to calculate a coastal zone mask and restrict the comparison of the products within the masked zone. Enabling this option would likely exclude pixels outside coastal zone (e.g. inland or deep ocean), but require more time to process. By default this parameter is not set to accelerate data query and loading. 
* `set_resolution`: Spatial resolution in metres to query all products. By default this parameter is not set, so the original resolutions of the products will be used, i.e. 30 m, 20 m and 10 m for Landsat, Sentinel-1 and Sentinel-2 respectively.  
* `set_product`: Set this to only query and load a pre-selected product, i.e. 'ls' for Landsat,'s1' for Sentinel-1,'s2' for Sentinel-2 or 'ls_s2' for merged/stacked Landsat and Sentinel-2 data. By setting this parameter no other products will be queried or compared.  

In [ ]:
ds_selected,product_name=load_best_available_ds(dc, lat_range, lon_range, time_range, time_step)


None of the available products are pre-selected, querying and compare all products...

Querying and loading Landsat data...
Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls8_sr
    ls9_sr
Applying morphological filters to pq mask [('opening', 1), ('dilation', 2)]
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Returning 86 time steps as a dask array

Querying and Sentinel-2 data...
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Applying morphological filters to pq mask [('opening', 1), ('dilation', 2)]
Applying pixel quality/cloud mask
Returning 578 time steps as a dask array

Querying and loading Sentinel-1 data...
Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Returning 191 time steps as a dask array

No coastal masking applied

Calculating number and frequency of valid observations...


/usr/local/lib/python3.10/dist-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


Note: Above function is intended to provide some guidance on selecting the product. It may take a few minutes to finish.  

Once the load is complete, examine the data by printing it in the next cell.
The `Dimensions` argument revels the number of time steps in the data set, as well as the number of pixels in the `x` (longitude) and `y` (latitude) dimensions.

In [10]:
print(product_name,ds_selected)

s2 <xarray.Dataset>
Dimensions:      (time: 578, y: 890, x: 875)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-02T07:22:04 ... 2021-12-30T07:...
  * y            (y) float64 -1.367e+06 -1.367e+06 ... -1.376e+06 -1.376e+06
  * x            (x) float64 3.582e+05 3.582e+05 3.582e+05 ... 3.67e+05 3.67e+05
    spatial_ref  int32 32638
Data variables:
    green        (time, y, x) float32 dask.array<chunksize=(1, 890, 875), meta=np.ndarray>
    swir_1       (time, y, x) float32 dask.array<chunksize=(1, 890, 875), meta=np.ndarray>
Attributes:
    crs:           epsg:32638
    grid_mapping:  spatial_ref


For this example notebook, you would find that Sentinel-2 was choosen through above function, as it has the higher mean number and frequency of valid observations compared to Landsat across the scene and within each time step. This may be expected at most of the locations. Meanwhile, in the above function, preference was given to Sentinel-2 and Landsat over Sentinel-1. This is in consideration of the findings that land and water are sometimes hard to be distinguished on SAR backscattering images; both can be low in values especially on smooth and flat beaches. However, if your query date range is before 2018, only Landsat data will be queried and loaded by the above function due to the limited availability of Sentinel data.

### Plot example timestamp
Here we can plot an example image at the selected location at a given timestamp. To visualise Landsat and Sentinel-2 data, use the pre-loaded `rgb` utility function to plot a true colour image for a given timestamp. White areas indicate where clouds or other invalid pixels in the image have been masked. If the selected product is Sentinel-1, we plot the vh band as an example.

In [11]:
# Set the timesteps to visualise
timestamp = 10
if (product_name=='ls') or (product_name=='s2'):
    # Generate RGB plots at each timestep
    rgb(ds_selected, index=timestamp, percentile_stretch=[0, 0.999])
else:
    ds_s1['vh'].isel(time=timestamp).plot.imshow(robust=True)

ValueError: rgb() bands do not match band names in dataset. Note the default rgb() bands are ['red', 'green', 'blue'].


Change the value for `timestamp` and re-run the cell to plot a different timestamp (timestamps are numbered from `0` to `n_time - 1` where `n_time` is the total number of timestamps; see the `time` listing under the `Dimensions` category in the dataset print-out above).


In [19]:
# ds_selected=ds_s1 # uncomment this line to choose Sentinel-1 product

## Process selected data and generate annual composites

### Model tide height  

For each satellite timestep, we use the `pixel_tides` function to model tide heights into a low-resolution 5 x 5 km grid (matching resolution of the [FES2014 tidal model](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes/description-fes2014.html)), then reprojects modelled tides into the spatial extent of our satellite image. We add this new data as a new variable in our satellite dataset to allow each satellite pixel to be analysed and filtered/masked based on the tide height at the exact moment of satellite image acquisition. 

In [19]:
ds_selected["tide_m"], tides_lowres = pixel_tides(ds_selected, resample=True)

Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array


100%|██████████| 578/578 [00:03<00:00, 171.63it/s]


Based on the entire time-series of tide heights, we can compute the max and min satellite-observed tide height for each pixel, then calculate tide cutoffs used to restrict our data to satellite observations centred over mid-tide (0 m Above Mean Sea Level) using the tide_cutoffs function:

In [ ]:
# Determine tide cutoff
tide_cutoff_min, tide_cutoff_max = tide_cutoffs(
    ds_selected, tides_lowres, tide_centre=0.0
)

With the tide cutoffs calculated for all pixels, we now only keep observations within the tide height cutoff ranges. We also want to drop time-steps with no pixels within the cutoff ranges:

In [ ]:
tide_bool = (ds_selected.tide_m >= tide_cutoff_min) & (
    ds_selected.tide_m <= tide_cutoff_max
)
ds_selected = ds_selected.sel(time=tide_bool.sum(dim=["x", "y"]) > 0)

# Apply mask, and load in corresponding tide masked data
ds_selected = ds_selected.where(tide_bool)
print(ds_selected)

### Process selected data
To extract shoreline locations, we need to be able to seperate water from land in our study area. 
To do this, for Landsat or Sentinel-2 we can calculate a water index called the `Modified Normalised Difference Water Index`, or MNDWI. 
This index uses the ratio of green and Shortwave-Infrared (SWIR) radiation to identify the presence of water [(Xu 2006)](https://doi.org/10.1080/01431160600589179). 
The formula is:

$$
\begin{aligned}
\text{MNDWI} &= \frac{(\text{Green} - \text{SWIR})}{(\text{Green} + \text{SWIR})}
\end{aligned}
$$

where `Green` is the green band and `SWIR` is the SWIR band. 

For Sentinel-1 data, as radar observations appear speckly due to random interference of coherent signals from target scatters, we may want to implement speckle filtering using Lee filter, which is one of the popular adaptive speckle filters that takes into account local homogeneity. Besides, it is often useful to convert the backscatter to decible (dB) for analysis. Backscatter in dB unit has a more symmetric noise profile and less skewed value distribution for easier statistical evaluation.  


In [20]:
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    
    return img_output

In [21]:
if product_name=='ls':
    # Calculate the water index
    ds_selected = calculate_indices(ds_selected, index='MNDWI', satellite_mission='ls')
elif (product_name=='s2') or (product_name=='ls_s2'):
    # Calculate the water index
    ds_selected = calculate_indices(ds_selected, index='MNDWI', satellite_mission='s2')
else:
    # The lee filter above doesn't handle null values
    # We therefore set null values to 0 before applying the filter
    valid = np.isfinite(ds_selected)
    ds_selected = ds_selected.where(valid, 0)

    if filter_size is None: # don't do speckle filtering
        ds_selected["filtered_vh"]=ds_selected["vh"]
    else: # do filtering
        # Create a new entry in dataset corresponding to filtered VV and VH data
        ds_selected["filtered_vh"] = ds_selected.vh.groupby("time").apply(lee_filter, size=filter_size)

    # Null pixels should remain null
    ds_selected['filtered_vh'] = ds_selected.filtered_vh.where(ds_selected.filtered_vh!=0,np.nan)

    # Scale to plot data in decibels
    ds_selected['filtered_vh'] = 10 * np.log10(ds_selected.filtered_vh)

### Combine observations into noise-free summary images
Individual remote sensing images can be affected by noisy data, e.g. effects of wind on the water. To produce cleaner images that can be compared more easily across time, we can create 'summary' images or composites that combine multiple images into one image to reveal the median or 'typical' appearance of the landscape for a certain time period. In this case, we use the median as the summary statistic because it prevents strong outliers from skewing the data, which would not be the case if we were to use the mean.

In the code below, we take the time series of images and combine them into single images for each `time_step`. For example, if `time_step = '2Y'`, the code will produce one new image for each two-year period in the dataset. This step can take **several minutes to load** if the study area is large.

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

In [ ]:
# Combine into summary images by `time_step`
if (product_name=='ls')or(product_name=='s2')or(product_name=='ls_s2'):
    var='MNDWI'
else:
    var='filtered_vh'

ds_summaries = (ds_selected[[var]]
                 .resample(time=time_step)
                 .median('time')
                 .compute()
                )
# Rename time attribute as year
ds_summaries['time'] = ds_summaries.time.dt.year
ds_summaries = ds_summaries.rename(time='year')

# Plot the output summary images
ds_summaries[var].plot(col='year',
                       cmap='RdBu',
                       col_wrap=2,
                       robust=True, size=10)
plt.show()



When it comes to interpreting the index, **High values (greater than 0, blue colours) typically represent water pixels**, while **low values (less than 0, red colours) represent land**. 

In [23]:
# Shut down Dask client now that we have processed the data we need
client.close()

## Extract shorelines and calculate rates of coastal change 
### Extract shorelines from imagery
We now want to extract an accurate shoreline for each of the summary images above. The code below identifies the boundary between land and water by tracing a line along pixels with a given threshold value. For Landsat and Sentinel-2 images, we use a water index value of `0`. 

For Sentinel-1 images, the threshold could be determined either through simple automatic thresholding, or using a more complicated supervised classification method. In this notebook we use the `threshold_minimum` function, which computes the histogram for all backscatter values, smooths it until there are only two maxima and find the minimum in between as the threshold. 

We use the `subpixel_contours` function to identify the boundary between land and water by tracing a line along pixels with the previously identified threshold value. It returns a vector file with one line for each time step:


In [25]:
if (product_name=='ls') or (product_name=='s2') or (product_name=='ls_s2'):
    threshold=0
else:
    threshold= threshold_minimum(ds_selected[var].values[~np.isnan(ds_selected[var].values)])

contour_gdf = subpixel_contours(da=ds_summaries[var],
                                z_values=threshold,
                                dim='year',
                                crs=ds_summaries.geobox.crs,
                                output_path='annual_shorelines_{}.geojson'.format(product_name),
                                min_vertices=15)
contour_gdf=contour_gdf.set_index('year')
# Preview shoreline data
contour_gdf

,geometry
year,
2018-12-31,"MULTILINESTRING ((358789.935 -1366660.000, 358..."
2019-12-31,"MULTILINESTRING ((358773.303 -1366660.000, 358..."
2020-12-31,"MULTILINESTRING ((358780.963 -1366660.000, 358..."
2021-12-31,"MULTILINESTRING ((358782.385 -1366660.000, 358..."


### Plot resampled shorelines on an interactive map
The next cell provides an interactive map with an overlay of the shorelines identified in the previous cell. Run it to view the map (this step can take **several minutes to load** if the study area is large).

Zoom in to the map below to explore the resulting set of shorelines. 
Older shorelines are coloured in black, and more recent shorelines in yellow.
Hover over the lines to see the time period for each shoreline printed above the map.
Using this data, we can easily identify areas of coastline or rivers that have changed significantly over time, or areas that have remained stable over the entire time period. 


In [26]:
# Plot shorelines on interactive map
contour_gdf.reset_index().explore(
    column='year',
    cmap='inferno',
    tiles=
    'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='ESRI WorldImagery')

### Calculate rates of coastal change
To identify parts of the coastline that are changing rapidly, we can use our annual shoreline data to calculate rates of coastal change in metres per year.
This can be particularly useful to reveal hotspots of coastal retreat (e.g. erosion), or hotspots of coastal growth.

To do this, we first need to create a set of evenly spaced points at every 20 metres along the most recent shoreline in our dataset. 
These points will be used to plot rates of coastal change across our study area.

In [ ]:
# Extract points at every 30 metres along the most recent shoreline
points_gdf = points_on_line(contour_gdf, index='2021', distance=20)
points_gdf.plot()

Now that we have a set of modelling points, we can measure distances from each of the points to each annual shoreline. 
This gives us a table of distances, where negative values (e.g. `-6.5`) indicate that an annual shoreline was located inland of our points, and positive values (e.g. `2.3`) indicate a shoreline was located towards the ocean. Because our points were created along our most recent 2021 shoreline, distances for 2021 will always have a distance of 0 m.

In [ ]:
# For each 30 m-spaced point, calculate the distance from
# the most recent 2021 shoreline to each other annual shoreline
# in the datasets.
points_gdf = annual_movements(points_gdf,
                              contours_gdf=contour_gdf,
                              yearly_ds=ds_summaries,
                              baseline_year=2021,
                              water_index=var)
points_gdf

Finally, we can calculate annual rates of coastal change (in metres per year) using linear regression. 
This will add several new columns to our table:

* `rate_time`: Annual rates of change (in metres per year) calculated by linearly regressing annual shoreline distances against time (excluding outliers; see `outl_time`). Negative values indicate retreat and positive values indicate growth. 
* `sig_time`: Significance (p-value) of the linear relationship between annual shoreline distances and time. Small values (e.g. p-value < 0.01 or 0.05) may indicate a coastline is undergoing consistent coastal change through time. 
* `se_time`: Standard error (in metres) of the linear relationship between annual shoreline distances and time. This can be used to generate confidence intervals around the rate of change given by rate_time (e.g. 95% confidence interval = `se_time * 1.96`)
* `outl_time`: Individual annual shoreline are noisy estimators of coastline position that can be influenced by environmental conditions (e.g. clouds, breaking waves, sea spray) or modelling issues (e.g. poor tidal modelling results or limited clear satellite observations). To obtain reliable rates of change, outlier shorelines are excluded using a robust Median Absolute Deviation outlier detection algorithm, and recorded in this column. 

In [ ]:
# Calculate rates of change using linear regression
points_gdf = calculate_regressions(points_gdf=points_gdf,
                                   contours_gdf=contour_gdf)
points_gdf

### Plot rates of coastal change on an interactive map
Now that we have calculated rates of coastal change, we can plot these on an interactive map to identify parts of the coastline that are retreating or growing over time.

When the map appears below, hover your mouse over the coloured dots that appear along the coastline for a summary of recent coastal change at those locations. 
Red dots represent locations that are retreating (e.g. erosion), and blue dots represent locations that are growing.

![coastal_change.jpg](../Supplementary_data/Coastal_erosion/coastal_change.jpg)

In [ ]:
# Add human-friendly label for plotting
points_gdf['Coastal change'] = points_gdf.apply(
    lambda x:
    f'<h4>This coastline has {"<b>retreated</b>" if x.rate_time < 0 else "<b>grown</b>"} '
    f'by</br><b>{x.rate_time:.2f} m (±{x.se_time:.1f}) per year</b> since '
    f'<b>{contour_gdf.index[0]}</b></h4>',
    axis=1)
points_gdf.loc[points_gdf.sig_time > 0.05, 'Coastal change'] = (
    f'<h4>No significant trend of retreat or growth)</h4>')

# Add annual shorelines to map
m = contour_gdf.reset_index().explore(
    column='year',
    cmap='inferno',
    tiles=
    'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    tooltip=False,
    style_kwds={'opacity': 0.5},
    attr='ESRI WorldImagery')

# Add rates of change to map
points_gdf.explore(
    m=m,
    column='rate_time',
    vmin=-5,
    vmax=5,
    tooltip='Coastal change',
    cmap='RdBu',
)

> **Important note:** This notebook may produce misleading rates of change for non-coastal waterbodies that might fluctuate naturally year-by-year.
The full [Digital Earth Africa Coastlines repository](https://github.com/digitalearthafrica/deafrica-coastlines.git) contains additional methods for producing more accurate rates of change by cleaning and filtering annual shoreline data to focus only on coastal shorelines.

### Export rates of change to file
Finally, we can export our output rates of change file so that it can be loaded in GIS software (e.g. ESRI ArcGIS or QGIS).

In [ ]:
points_gdf.to_crs('EPSG:4326').to_file(output_path.replace('annual_shorelines','rates_of_changes'))

## Next steps
When you are done, return to the "Set up analysis" cell, modify some values (e.g. `time_range`, `time_step` or `lat`/`lon`) and rerun the analysis.
If you're going to change the location, you'll need to make sure Sentinel-1 data is available for the new location, which you can check at the [DE Africa Explorer](https://explorer.digitalearth.africa/products/). 

For more information about the method behind this notebook, read the scientific paper:
> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')